# Hyperparameter Tuning

## Imports

In [1]:
import os
import random
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from triplet_utils import *
from utils import *
from torch.utils.data import random_split
from triplet_utils import train_and_evaluate_triplet


### Set up the model

In [2]:
# TripletModel definition
class TripletModel(nn.Module):
    def __init__(self):
        super(TripletModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, 128)  # Adjust output size for embedding

    def forward(self, x):
        # Get the embedding directly from the base model
        return self.base_model(x)  # Call the model with input x

class TripletClassifier(nn.Module):
    def __init__(self, triplet_head, output_size=1):
        super(TripletClassifier, self).__init__()
        self.triplet_head = triplet_head
        self.classifier = nn.Linear(self.triplet_head.base_model.fc.out_features, output_size)  # Binary classification (logits)

    def forward(self, x):
        embedding = self.triplet_head(x)  # Output of base model
        class_output = self.classifier(embedding)  # Output for classification
        return embedding, class_output

### Set your dataset directory

In [3]:
# Load data
data_dir = '../../data/BiteCount/salient_poses/'

# Define transformations for training set and validation set
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to square input
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing to square input
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
dataset = TripletDataset(data_dir, transform=train_transforms)
dataset_size = len(dataset)
class_names = dataset.classes

# Hyperparameters
batch_size = 64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_epochs = 50
patience = 5
alpha = 0.5

# Placeholder to store metrics across folds
val_accuracies = []
val_f1_scores = []
precisions = []
recalls = []
roc_aucs = []

### Set your model and parameters

In [4]:
def custom_collate(batch):
    # Unpack the batch
    anchors, positives, negatives, labels = zip(*batch)

    # Stack tensors to create batches
    anchors = torch.stack(anchors)
    positives = torch.stack(positives)
    negatives = torch.stack(negatives)

    # Convert labels to a tensor
    labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)

    return anchors, positives, negatives, labels

In [5]:
import torch.nn.functional as F

# Custom Triplet Loss without margin
def triplet_loss_no_margin(anchor, positive, negative):
    # Calculate the pairwise distances
    pos_dist = F.pairwise_distance(anchor, positive)
    neg_dist = F.pairwise_distance(anchor, negative)
    
    # Compute the triplet loss without a margin
    loss = torch.clamp(pos_dist - neg_dist, min=0.0)
    
    # Return the mean of the loss over the batch
    return loss.mean()

## Tune the model

In [6]:
import itertools
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import KFold
import torch.optim as optim
import torch.nn as nn
import numpy as np

# Define the range of hyperparameters to test
alphas = [0.3, 0.5, 0.7, 0.9]  # Different values for alpha
learning_rates = [0.001, 0.01, 0.1]  # Example learning rates
momentums = [0.5, 0.8, 0.9]  # Different momentum values

# Store the best results
best_val_acc = 0.0
best_hyperparams = {}
best_val_loss = 1e10

# Create a search space using itertools
search_space = itertools.product(alphas, learning_rates, momentums)

# Create KFold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for alpha, lr, momentum in search_space:
    print(f"Training alpha: {alpha}, learning_rate: {lr}, momentum: {momentum}")

    fold_val_accs = []
    fold_val_losses = []

    for fold, (train_index, val_index) in enumerate(kf.split(range(dataset_size))):
        print(f"Fold {fold + 1}")

        # Create training and validation datasets using the indices from k-fold
        train_dataset = [dataset[i] for i in train_index]
        val_dataset = [dataset[i] for i in val_index]

        # Optionally, create DataLoader for training and validation datasets
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

        # Initialize TripletModel
        triplet_model = TripletModel()
        model = TripletClassifier(triplet_model)
        model = model.to(device)

        # Define loss functions and optimizer
        criterion_classification = nn.BCEWithLogitsLoss()  # Binary cross-entropy for classification
        criterion_triplet = triplet_loss_no_margin # Use margin from the search space
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # Optimizer with dynamic lr and momentum
        patience_counter = 0
        best_val_acc = 0

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0

            for batch in train_loader:
                anchor, positive, negative, labels = batch
                anchor, positive, negative, labels = anchor.to(device), positive.to(device), negative.to(device), labels.to(device).unsqueeze(1).float() 

                # Forward pass
                anchor_embedding, anchor_output = model(anchor)
                positive_embedding, positive_output = model(positive)
                negative_embedding, negative_output = model(negative)

                # Calculate losses
                classification_loss = criterion_classification(anchor_output, labels)
                triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)

                # Combine losses
                loss = alpha * classification_loss + (1 - alpha) * triplet_loss
                running_loss += loss.item()

                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            avg_loss = running_loss / len(train_loader)
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

            # Validation step
            model.eval()
            val_preds, val_labels = [], []
            val_loss = 0.0

            with torch.no_grad():
                for batch in val_loader:
                    anchor, positive, negative, labels = batch
                    anchor, positive, negative, labels = anchor.to(device), positive.to(device), negative.to(device), labels.to(device).unsqueeze(1).float() 

                    anchor_embedding, anchor_output = model(anchor)
                    positive_embedding, positive_output = model(positive)
                    negative_embedding, negative_output = model(negative)

                    # Compute validation loss
                    classification_loss = criterion_classification(anchor_output, labels)
                    triplet_loss = criterion_triplet(anchor_embedding, positive_embedding, negative_embedding)
                    val_loss += (classification_loss + triplet_loss).item()

                    val_preds.append(torch.sigmoid(anchor_output).cpu().numpy())
                    val_labels.append(labels.cpu().numpy())

            avg_val_loss = val_loss / len(val_loader)
            val_preds = np.concatenate(val_preds)
            val_labels = np.concatenate(val_labels)

            # Calculate validation metrics
            val_acc = accuracy_score(val_labels, (val_preds > 0.5).astype(int))
            val_f1 = f1_score(val_labels, (val_preds > 0.5).astype(int))
            precision = precision_score(val_labels, (val_preds > 0.5).astype(int))
            recall = recall_score(val_labels, (val_preds > 0.5).astype(int))
            roc_auc = roc_auc_score(val_labels, val_preds)

            print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}, "
                  f"Precision: {precision:.4f}, Recall: {recall:.4f}, ROC AUC: {roc_auc:.4f}")

            # Early stopping and best model saving
            if best_val_acc < val_acc:
                best_val_loss = avg_val_loss
                best_val_acc = val_acc
                best_hyperparams = {'alpha': alpha, 'learning_rate': lr, 'momentum': momentum}
                patience_counter = 0
                torch.save(model.state_dict(), f'triplet_classifier_best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping triggered.")
                    break

        # Store results for this fold
        fold_val_accs.append(val_acc)
        fold_val_losses.append(avg_val_loss)

    # Calculate average metrics across folds
    avg_fold_val_acc = np.mean(fold_val_accs)
    avg_fold_val_loss = np.mean(fold_val_losses)
    print(f"Average Validation Accuracy for alpha: {alpha}, lr: {lr}, momentum: {momentum}: {avg_fold_val_acc:.4f}")


Training alpha: 0.3, learning_rate: 0.001, momentum: 0.5
Fold 1
Epoch [1/50], Loss: 1.1965
Validation Loss: 2.0703, Accuracy: 0.4375, F1 Score: 0.6018, Precision: 0.4304, Recall: 1.0000, ROC AUC: 0.4322
Epoch [2/50], Loss: 0.9283
Validation Loss: 1.7870, Accuracy: 0.4500, F1 Score: 0.6071, Precision: 0.4359, Recall: 1.0000, ROC AUC: 0.4425
Epoch [3/50], Loss: 0.7484
Validation Loss: 1.5978, Accuracy: 0.4375, F1 Score: 0.6018, Precision: 0.4304, Recall: 1.0000, ROC AUC: 0.4808
Epoch [4/50], Loss: 0.6310
Validation Loss: 1.4853, Accuracy: 0.3875, F1 Score: 0.5505, Precision: 0.4000, Recall: 0.8824, ROC AUC: 0.4885
Epoch [5/50], Loss: 0.5353
Validation Loss: 1.4237, Accuracy: 0.3875, F1 Score: 0.5505, Precision: 0.4000, Recall: 0.8824, ROC AUC: 0.4757
Epoch [6/50], Loss: 0.4824
Validation Loss: 1.3570, Accuracy: 0.3875, F1 Score: 0.5333, Precision: 0.3944, Recall: 0.8235, ROC AUC: 0.4904
Epoch [7/50], Loss: 0.4239
Validation Loss: 1.3266, Accuracy: 0.3875, F1 Score: 0.5333, Precision: 0.3

/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [3/50], Loss: 0.3392
Validation Loss: 1.3368, Accuracy: 0.5063, F1 Score: 0.4179, Precision: 0.5600, Recall: 0.3333, ROC AUC: 0.5097
Epoch [4/50], Loss: 0.2924
Validation Loss: 1.3532, Accuracy: 0.5823, F1 Score: 0.4590, Precision: 0.7368, Recall: 0.3333, ROC AUC: 0.6030
Epoch [5/50], Loss: 0.2772
Validation Loss: 1.3351, Accuracy: 0.5316, F1 Score: 0.5432, Precision: 0.5641, Recall: 0.5238, ROC AUC: 0.5721
Epoch [6/50], Loss: 0.2543
Validation Loss: 1.3793, Accuracy: 0.5823, F1 Score: 0.6118, Precision: 0.6047, Recall: 0.6190, ROC AUC: 0.6088
Epoch [7/50], Loss: 0.2280
Validation Loss: 1.5769, Accuracy: 0.5949, F1 Score: 0.6444, Precision: 0.6042, Recall: 0.6905, ROC AUC: 0.5476
Epoch [8/50], Loss: 0.2286
Validation Loss: 1.2905, Accuracy: 0.5949, F1 Score: 0.6190, Precision: 0.6190, Recall: 0.6190, ROC AUC: 0.7001
Epoch [9/50], Loss: 0.1840
Validation Loss: 1.2851, Accuracy: 0.7215, F1 Score: 0.7609, Precision: 0.7000, Recall: 0.8333, ROC AUC: 0.7439
Epoch [10/50], Loss: 0.1707

/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [2/50], Loss: 0.4855
Validation Loss: 1.0076, Accuracy: 0.4250, F1 Score: 0.5965, Precision: 0.4250, Recall: 1.0000, ROC AUC: 0.3836
Epoch [3/50], Loss: 0.2545
Validation Loss: 0.7821, Accuracy: 0.4250, F1 Score: 0.5965, Precision: 0.4250, Recall: 1.0000, ROC AUC: 0.4258
Epoch [4/50], Loss: 0.2251
Validation Loss: 0.7826, Accuracy: 0.5750, F1 Score: 0.0000, Precision: 0.0000, Recall: 0.0000, ROC AUC: 0.5345


...

Epoch [4/50], Loss: 0.6701
Validation Loss: 1.3356, Accuracy: 0.4684, F1 Score: 0.0000, Precision: 0.0000, Recall: 0.0000, ROC AUC: 0.4659


/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch [5/50], Loss: 0.6340
Validation Loss: 0.7699, Accuracy: 0.5316, F1 Score: 0.6942, Precision: 0.5316, Recall: 1.0000, ROC AUC: 0.5090
Epoch [6/50], Loss: 0.6386
Validation Loss: 0.8011, Accuracy: 0.4684, F1 Score: 0.0000, Precision: 0.0000, Recall: 0.0000, ROC AUC: 0.5187
Early stopping triggered.
Average Validation Accuracy for alpha: 0.9, lr: 0.1, momentum: 0.9: 0.4913


/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


KeyError: 'margin'

In [7]:
# Save the results to a text file
with open('best_hyperparameters_tmm.txt', 'w') as f:
    f.write(f"Best validation loss: {best_val_loss:.4f}\n")
    f.write(f"Best accuracy: {best_val_acc:.4f}\n")
    f.write(f"Best hyperparameters:\n")
    f.write(f"  Alpha: {best_hyperparams['alpha']}\n")
    f.write(f"  Learning Rate: {best_hyperparams['learning_rate']}\n")
    f.write(f"  Momentum: {best_hyperparams['momentum']}\n")

print("Results saved to 'best_hyperparameters_tmm.txt'")


Results saved to 'best_hyperparameters_tmm.txt'
